In [1]:
from sentence_transformers import SentenceTransformer

model_name = "hiieu/halong_embedding"

model = SentenceTransformer('hiieu/halong_embedding')


/home/mahdi/text_embedding/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_from_disk
ds = load_from_disk('APN_dataset')
print(ds)                # see if it’s a Dataset or DatasetDict
print(ds.column_names)

Dataset({
    features: ['anchor', 'positive', 'negetive'],
    num_rows: 686
})
['anchor', 'positive', 'negetive']


In [4]:
# print first 5 examples
for ex in ds.select(range(5)):
    print(ex)


{'anchor': 'عقد مرابحه یک نوع عقد بانکی است که در آن بانک برای مبلغ وام\u200cدهی، به برداشتن شخص قابل امتیاز تشخیص می\u200cدهد که باید برای این مقدار دریافت شده چقدر سود بپردازد. این اطلاعات به صورت منظم و با روشنی تعیین می\u200cشود، که نتیجه آن شفافیت در قرارداد است.', 'positive': 'عقد مرابحه چیست و آیا شفاف بودن قرارداد اصلی در این نوع عقد با اشاره به مبلغ سود است؟', 'negetive': 'چه نوع از وام\u200cهای ازدواجی توصیه می\u200cشود و آیا آن\u200cها در این منبع تحت پوشش هستند؟'}
{'anchor': 'بله، به نقطه تاس۱۰ توجه کنید: «امکان ضمانت متقابل انواع وام\u200cها در بانک ملت وجود ندارد».', 'positive': 'آیا امکان ضمانت متقابل در وام\u200cهای مختلف در بانک ملت وجود دارد؟', 'negetive': 'نحوه محاسبه توان بازپرداخت چگونه است؟'}
{'anchor': 'قرض الحسنه یعنی وامی است که بدون سود و بهره داده می\u200cشود و فقط اصل پول باید برگردانده شود.', 'positive': 'قرض الحسنه به معنای کدام نوع وام است؟', 'negetive': 'وام های ازدواج و فرزندآوری در طبقه\u200cبندی معمول تکلیفی قرار می\u200cگیرند یا نه؟'}
{'anchor': 'وثی

In [5]:
splits = ds.train_test_split(test_size=0.1, seed=42)
train_ds = splits['train']
dev_ds   = splits['test']


In [6]:
# 1. Rename the typo’d column
train_ds = train_ds.rename_column("negetive", "negative")
dev_ds   = dev_ds.rename_column("negetive", "negative")

In [7]:
from sentence_transformers import InputExample

# assume `train_ds` is your HF Dataset with columns ['anchor','positive','negative']

train_examples = [
    InputExample(
        texts=[item['anchor'], item['positive'], item['negative']]
    )
    for item in train_ds  # iterates over all examples
]

# if you also split out dev_ds:
dev_examples = [
    InputExample(
        texts=[item['anchor'], item['positive'], item['negative']]
    )
    for item in dev_ds
]


In [8]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses

model = SentenceTransformer('hiieu/halong_embedding')

train_loader = DataLoader(train_examples, shuffle=True, batch_size=16)
# train_loss   = losses.TripletLoss(model=model)
train_loss = losses.TripletLoss(
    model=model,
    triplet_margin=0.5,               # you can tune this margin
    distance_metric=losses.SiameseDistanceMetric.COSINE
)
# (optional) set up a triplet evaluator on dev_examples, or skip if you don’t have one
# from sentence_transformers.evaluation import TripletEvaluator
# triplet_evaluator = TripletEvaluator.from_input_examples(dev_examples, name='persian-triplet')

model.fit(
    train_objectives=[(train_loader, train_loss)],
    epochs=3,
    warmup_steps=100,
    # evaluator=triplet_evaluator,
    # evaluation_steps=len(train_loader),
    output_path='./halong_persian_triplet'
)


OutOfMemoryError: CUDA out of memory. Tried to allocate 734.00 MiB. GPU 0 has a total capacity of 7.77 GiB of which 450.25 MiB is free. Process 229497 has 5.60 GiB memory in use. Including non-PyTorch memory, this process has 1.24 GiB memory in use. Of the allocated memory 1.04 GiB is allocated by PyTorch, and 55.27 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)